In [17]:
import cv2
import numpy as np
from ultralytics import YOLO
import time
import threading
import os

In [18]:
# ===================== Settings =====================
CAMERA_INDEX = 0
CONFIDENCE_THRESHOLD = 0.5
ALERT_COOLDOWN_S = 2.0
DANGER_ZONE_RATIO = 0.6  # Bottom 60% of frame is danger zone
DRAW_DEBUG = True
# =====================================================

class PedestrianDetector:
    def __init__(self):
        print("Initializing Pedestrian Detector...")
        
        # Load YOLO model
        try:
            self.model = YOLO("yolov8n.pt")
            print("YOLO model loaded successfully!")
        except Exception as e:
            print(f"Error loading YOLO model: {e}")
            raise
        
        # Alert system
        self.last_alert_time = 0
        self.alert_cooldown = ALERT_COOLDOWN_S
        
        # Detection history for stability
        self.detection_history = []
        self.history_size = 5
        
        # Alert sound function
        self.setup_alert_sound()
    
    def setup_alert_sound(self):
        """Setup alert sound system"""
        try:
            import winsound
            self.beep_func = lambda: winsound.Beep(2000, 500)
            print("Windows beep sound enabled")
        except ImportError:
            try:
                import pygame
                pygame.mixer.init()
                self.beep_func = lambda: pygame.mixer.Sound.play(pygame.mixer.Sound("beep.wav"))
                print("Pygame sound enabled")
            except ImportError:
                self.beep_func = lambda: print("BEEP! (No sound available)")
                print("No sound system available - using print alert")
    
    def is_in_danger_zone(self, bbox, frame_height):
        """Check if pedestrian is in the danger zone (bottom portion of frame)"""
        x1, y1, x2, y2 = bbox
        danger_zone_top = int(frame_height * (1 - DANGER_ZONE_RATIO))
        
        # Check if pedestrian's bottom is in danger zone
        return y2 > danger_zone_top
    
    def draw_danger_zone(self, frame):
        """Draw danger zone overlay"""
        h, w = frame.shape[:2]
        danger_zone_top = int(h * (1 - DANGER_ZONE_RATIO))
        
        # Semi-transparent red overlay
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, danger_zone_top), (w, h), (0, 0, 255), -1)
        cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
        
        # Danger zone line
        cv2.line(frame, (0, danger_zone_top), (w, danger_zone_top), (0, 0, 255), 3)
        cv2.putText(frame, "DANGER ZONE", (10, danger_zone_top - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    
    def alert_system(self):
        """Trigger alert (sound/visual)"""
        current_time = time.time()
        if current_time - self.last_alert_time >= self.alert_cooldown:
            self.last_alert_time = current_time
            print("🚨 ALERT: Pedestrian in danger zone!")
            
            # Play sound in separate thread to avoid blocking
            threading.Thread(target=self.beep_func, daemon=True).start()
    
    def detect_pedestrians(self, frame):
        """Detect pedestrians and check danger zone"""
        try:
            results = self.model(frame, conf=CONFIDENCE_THRESHOLD, verbose=False)
            
            pedestrians_in_danger = 0
            total_pedestrians = 0
            
            for result in results:
                boxes = result.boxes
                if boxes is not None:
                    for box in boxes:
                        # Check if it's a person (class 0 in COCO dataset)
                        if int(box.cls[0]) == 0:  # Person class
                            total_pedestrians += 1
                            
                            # Get bounding box coordinates
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
                            confidence = float(box.conf[0])
                            
                            # Check if in danger zone
                            if self.is_in_danger_zone((x1, y1, x2, y2), frame.shape[0]):
                                pedestrians_in_danger += 1
                                
                                # Draw danger alert
                                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                                cv2.putText(frame, f"DANGER! {confidence:.2f}", (x1, y1-10), 
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                                
                                # Trigger alert
                                self.alert_system()
                            else:
                                # Safe pedestrian
                                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                cv2.putText(frame, f"Safe {confidence:.2f}", (x1, y1-10), 
                                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            return total_pedestrians, pedestrians_in_danger
            
        except Exception as e:
            print(f"Detection error: {e}")
            return 0, 0
    
    def run_detection(self):
        """Main detection loop"""
        print("Starting camera...")
        cap = cv2.VideoCapture(CAMERA_INDEX)
        
        # Set camera properties for better performance
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        cap.set(cv2.CAP_PROP_FPS, 30)
        
        if not cap.isOpened():
            print("Error: Could not open webcam")
            print("Trying alternative camera indices...")
            
            # Try different camera indices
            for i in range(1, 5):
                cap = cv2.VideoCapture(i)
                if cap.isOpened():
                    print(f"Camera found at index {i}")
                    break
                cap.release()
            
            if not cap.isOpened():
                raise RuntimeError("Could not open any camera")
        
        print("Camera opened successfully!")
        print("Pedestrian Detection System Started")
        print("Press 'q' to quit, 's' to save screenshot")
        
        frame_count = 0
        
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("Failed to read frame from camera")
                    break
                
                frame_count += 1
                
                # Draw danger zone
                if DRAW_DEBUG:
                    self.draw_danger_zone(frame)
                
                # Detect pedestrians
                total_peds, danger_peds = self.detect_pedestrians(frame)
                
                # Status display
                status_color = (0, 255, 0) if danger_peds == 0 else (0, 0, 255)
                cv2.putText(frame, f"Pedestrians: {total_peds} | Danger: {danger_peds}", 
                           (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)
                
                # Instructions
                cv2.putText(frame, "Press 'q' to quit", (10, frame.shape[0] - 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Show frame
                cv2.imshow("Pedestrian Detection System", frame)
                
                # Check for key presses
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    print("Quitting...")
                    break
                elif key == ord('s'):
                    # Save screenshot
                    filename = f"pedestrian_detection_{int(time.time())}.jpg"
                    cv2.imwrite(filename, frame)
                    print(f"Screenshot saved: {filename}")
                
                # Print status every 30 frames
                if frame_count % 30 == 0:
                    print(f"Frame {frame_count}: Pedestrians={total_peds}, Danger={danger_peds}")
        
        except KeyboardInterrupt:
            print("\nInterrupted by user")
        except Exception as e:
            print(f"Error in detection loop: {e}")
        finally:
            print("Cleaning up...")
            cap.release()
            cv2.destroyAllWindows()
            print("System stopped")

# ===================== Usage =====================
if __name__ == "__main__":
    try:
        # Create detector
        detector = PedestrianDetector()
        
        # Run detection
        detector.run_detection()
        
    except Exception as e:
        print(f"Fatal error: {e}")
        print("Please check:")
        print("1. Camera is connected and not used by another application")
        print("2. Required packages are installed: ultralytics, opencv-python")
        print("3. Try running as administrator if on Windows")

Initializing Pedestrian Detector...
YOLO model loaded successfully!
Windows beep sound enabled
Starting camera...
Camera opened successfully!
Pedestrian Detection System Started
Press 'q' to quit, 's' to save screenshot
🚨 ALERT: Pedestrian in danger zone!
🚨 ALERT: Pedestrian in danger zone!
Frame 30: Pedestrians=2, Danger=2
🚨 ALERT: Pedestrian in danger zone!
Frame 60: Pedestrians=2, Danger=2
🚨 ALERT: Pedestrian in danger zone!
🚨 ALERT: Pedestrian in danger zone!
Frame 90: Pedestrians=2, Danger=2
🚨 ALERT: Pedestrian in danger zone!
Frame 120: Pedestrians=3, Danger=2
🚨 ALERT: Pedestrian in danger zone!
Frame 150: Pedestrians=2, Danger=1
🚨 ALERT: Pedestrian in danger zone!
Frame 180: Pedestrians=0, Danger=0
🚨 ALERT: Pedestrian in danger zone!
🚨 ALERT: Pedestrian in danger zone!
Frame 210: Pedestrians=1, Danger=1
🚨 ALERT: Pedestrian in danger zone!
Frame 240: Pedestrians=0, Danger=0
Frame 270: Pedestrians=0, Danger=0
🚨 ALERT: Pedestrian in danger zone!
Frame 300: Pedestrians=0, Danger=0
F